In [3]:
import requests
import os
# https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
response = requests.get("https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw/d72b9558a11523adbe13300b41321ecd93d331d3/document.txt")
target_file = "files/chapter_three.txt"
os.makedirs(os.path.dirname(target_file), exist_ok=True)
with open(target_file, "wb") as f:
    f.write(response.content)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
)
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embedding,
    cache_dir,
)

vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriver = vectorstore.as_retriever()